In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import joblib
import warnings
warnings.filterwarnings('ignore')

# Import XGBoost avec gestion d'erreur
try:
    from xgboost import XGBRegressor
    XGBOOST_AVAILABLE = True
    print("✅ XGBoost disponible")
except ImportError:
    XGBOOST_AVAILABLE = False
    print("⚠️ XGBoost non installé. Installez avec: pip install xgboost")

print("🚗 GetAround - Pipeline ML Simplifié")
print("=" * 50)

✅ XGBoost disponible
🚗 GetAround - Pipeline ML Simplifié


In [4]:
# ============================================================================
# ÉTAPE 1: CHARGEMENT DES DONNÉES
# ============================================================================
print("\n📁 ÉTAPE 1: CHARGEMENT DES DONNÉES")
print("-" * 30)

# Chargement du fichier CSV
df = pd.read_csv('../data/get_around_pricing_project.csv')

# Suppression colonne index si elle existe
if 'Unnamed: 0' in df.columns:
    df = df.drop('Unnamed: 0', axis=1)

print(f"✅ Données chargées: {df.shape[0]} voitures, {df.shape[1]} colonnes")
print(f"📊 Colonnes: {list(df.columns)}")

# Aperçu rapide
print(f"\n📋 Aperçu des données:")
df.head(3)


📁 ÉTAPE 1: CHARGEMENT DES DONNÉES
------------------------------
✅ Données chargées: 4843 voitures, 14 colonnes
📊 Colonnes: ['model_key', 'mileage', 'engine_power', 'fuel', 'paint_color', 'car_type', 'private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires', 'rental_price_per_day']

📋 Aperçu des données:


,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,140411,100,diesel,black,convertible,True,True,False,False,True,True,True,106
1,Citroën,13929,317,petrol,grey,convertible,True,True,False,False,False,True,True,264
2,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101


In [5]:
df.isna().sum()

model_key                    0
mileage                      0
engine_power                 0
fuel                         0
paint_color                  0
car_type                     0
private_parking_available    0
has_gps                      0
has_air_conditioning         0
automatic_car                0
has_getaround_connect        0
has_speed_regulator          0
winter_tires                 0
rental_price_per_day         0
dtype: int64

In [6]:
# ============================================================================
# ÉTAPE 2: NETTOYAGE DES DONNÉES (SIMPLE)
# ============================================================================
print("\n🧹 ÉTAPE 2: NETTOYAGE DES DONNÉES")
print("-" * 30)

# Vérification valeurs manquantes
print("🔍 Valeurs manquantes par colonne:")
missing = df.isnull().sum()
print(missing[missing > 0])

# Nettoyage simple des anomalies identifiées dans l'EDA
print("\n🔧 Corrections des anomalies:")

# 1. Kilométrage négatif → 0
negative_mileage = (df['mileage'] < 0).sum()
df.loc[df['mileage'] < 0, 'mileage'] = 0
print(f"   • Kilométrage négatif corrigé: {negative_mileage} cas")

# 2. Conversion variables booléennes
print(f"   • Conversion variables True/False en 0/1")
bool_columns = ['private_parking_available', 'has_gps', 'has_air_conditioning', 
                'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 
                'winter_tires']

for col in bool_columns:
    df[col] = df[col].astype(str).map({'True': 1, 'False': 0})

print(f"✅ Nettoyage terminé !")


🧹 ÉTAPE 2: NETTOYAGE DES DONNÉES
------------------------------
🔍 Valeurs manquantes par colonne:


Series([], dtype: int64)

🔧 Corrections des anomalies:
   • Kilométrage négatif corrigé: 1 cas
   • Conversion variables True/False en 0/1
✅ Nettoyage terminé !


In [7]:
# ============================================================================
# ÉTAPE 3: CRÉATION DE NOUVELLES VARIABLES (FEATURE ENGINEERING SIMPLE)
# ============================================================================
print("\n🔧 ÉTAPE 3: CRÉATION DE NOUVELLES VARIABLES")
print("-" * 30)

# Variable 1: Score d'équipements (0 à 7)
df['equipment_score'] = df[bool_columns].sum(axis=1)
print(f"   • equipment_score: somme des équipements (0-7)")

# Variable 2: Age estimé basé sur le kilométrage (hypothèse: 15k km/an)
df['estimated_age'] = df['mileage'] / 15000
df['estimated_age'] = df['estimated_age'].clip(upper=20)  # Max 20 ans
print(f"   • estimated_age: kilométrage/15000 (max 20 ans)")

# Feature 4: Marque premium (amélioration basée sur l'analyse)
luxury_brands = ['BMW', 'Mercedes-Benz', 'Audi', 'Porsche', 'Lexus', 'Jaguar', 'Ferrari', 'Lamborghini']
df['is_luxury_brand'] = df['model_key'].isin(luxury_brands).astype(int)
luxury_count = df['is_luxury_brand'].sum()
print(f"   • is_luxury_brand: {luxury_count} marques premium identifiées")

# Variable 5: Interaction puissance/âge (feature importante)
df['power_age_ratio'] = df['engine_power'] / (df['estimated_age'] + 1)
print(f"   • power_age_ratio: ratio puissance/âge pour capturer dépréciation")

print(f"✅ 5 nouvelles variables créées (vs 3 dans version précédente)")

# Aperçu des nouvelles variables
print(f"\n📊 Aperçu nouvelles variables:")
new_vars = ['equipment_score', 'estimated_age', 'is_luxury_brand', 'power_age_ratio']
print(df[new_vars].describe())


🔧 ÉTAPE 3: CRÉATION DE NOUVELLES VARIABLES
------------------------------
   • equipment_score: somme des équipements (0-7)
   • estimated_age: kilométrage/15000 (max 20 ans)
   • is_luxury_brand: 1396 marques premium identifiées
   • power_age_ratio: ratio puissance/âge pour capturer dépréciation
✅ 5 nouvelles variables créées (vs 3 dans version précédente)

📊 Aperçu nouvelles variables:
       equipment_score  estimated_age  is_luxury_brand  power_age_ratio
count      4843.000000    4843.000000      4843.000000      4843.000000
mean          3.376833       9.342302         0.288251        15.406012
std           1.645421       3.773768         0.452995        13.300691
min           0.000000       0.000000         0.000000         0.000000
25%           2.000000       6.860900         0.000000         9.064218
50%           3.000000       9.405333         0.000000        11.856770
75%           5.000000      11.679700         1.000000        17.040705
max           7.000000      20.

In [8]:
# ============================================================================
# ÉTAPE 4: PRÉPARATION DES DONNÉES POUR ML
# ============================================================================
print("\n⚙️ ÉTAPE 4: PRÉPARATION POUR ML")
print("-" * 30)

# Définition de la variable cible (target)
target = 'rental_price_per_day'
y = df[target]
print(f"🎯 Variable cible: {target}")
print(f"   • Min: {y.min()}€, Max: {y.max()}€, Moyenne: {y.mean():.1f}€")

# Features numériques
numeric_features = ['mileage', 'engine_power',
                  'private_parking_available', 'has_gps', 
                  'has_air_conditioning', 'automatic_car',
                  'has_getaround_connect', 'has_speed_regulator', 
                  'winter_tires']

# Features catégorielles
categorical_features = ['model_key', 'fuel', 'paint_color', 'car_type']

print(f"\n📊 Features sélectionnées (améliorées):")
print(f"   • Numériques: {numeric_features}")
print(f"   • Catégorielles: {categorical_features}")
print(f"   • Total features: {len(numeric_features) + len(categorical_features)}")

# Préparation des features numériques
X_numeric = df[numeric_features].copy()

# Préparation des features catégorielles avec Label Encoding (simple)
X_categorical = df[categorical_features].copy()

# Label Encoding pour les variables catégorielles
label_encoders = {}
for col in categorical_features:
    le = LabelEncoder()
    X_categorical[col] = le.fit_transform(X_categorical[col].astype(str))
    label_encoders[col] = le
    print(f"   • {col}: {len(le.classes_)} catégories encodées")

# Combinaison de toutes les features
X = pd.concat([X_numeric, X_categorical], axis=1)
feature_names = X.columns.tolist()

print(f"\n✅ Dataset ML prêt:")
print(f"   • Features: {len(feature_names)} variables")
print(f"   • Échantillons: {len(X)} voitures")
print(f"   • Features finales: {feature_names}")


⚙️ ÉTAPE 4: PRÉPARATION POUR ML
------------------------------
🎯 Variable cible: rental_price_per_day
   • Min: 10€, Max: 422€, Moyenne: 121.2€

📊 Features sélectionnées (améliorées):
   • Numériques: ['mileage', 'engine_power', 'private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']
   • Catégorielles: ['model_key', 'fuel', 'paint_color', 'car_type']
   • Total features: 13
   • model_key: 28 catégories encodées
   • fuel: 4 catégories encodées
   • paint_color: 10 catégories encodées
   • car_type: 8 catégories encodées

✅ Dataset ML prêt:
   • Features: 13 variables
   • Échantillons: 4843 voitures
   • Features finales: ['mileage', 'engine_power', 'private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires', 'model_key', 'fuel', 'paint_color', 'car_type']


In [9]:
# ============================================================================
# ÉTAPE 5: DIVISION TRAIN/TEST
# ============================================================================
print("\n📊 ÉTAPE 5: DIVISION TRAIN/TEST")
print("-" * 30)

# Division 80% train, 20% test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2, 
    random_state=42,
    shuffle=True
)

print(f"📈 Données d'entraînement: {len(X_train)} voitures")
print(f"📉 Données de test: {len(X_test)} voitures")
print(f"📊 Ratio: {len(X_train)/len(X)*100:.1f}% train, {len(X_test)/len(X)*100:.1f}% test")

# Vérification des distributions
print(f"\n🔍 Vérification distributions:")
print(f"   • Train - Prix moyen: {y_train.mean():.1f}€")
print(f"   • Test - Prix moyen: {y_test.mean():.1f}€")
print(f"   • Différence: {abs(y_train.mean() - y_test.mean()):.1f}€ (doit être faible)")


📊 ÉTAPE 5: DIVISION TRAIN/TEST
------------------------------
📈 Données d'entraînement: 3874 voitures
📉 Données de test: 969 voitures
📊 Ratio: 80.0% train, 20.0% test

🔍 Vérification distributions:
   • Train - Prix moyen: 121.4€
   • Test - Prix moyen: 120.6€
   • Différence: 0.8€ (doit être faible)


In [10]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [11]:
# ============================================================================
# ÉTAPE 6: Préprocessing des données
# ============================================================================
print("\n📏 ÉTAPE 6: PREPROCESSING")
print("-" * 30)


def create_pipeline():    
    # Preprocessing numérique
    numerical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Preprocessing catégoriel
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
    ])
    
    return ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ]
    )

# Création du preprocessing pipeline
preprocessor = create_pipeline()

# Fit et transform sur les données d'entraînement
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Vérification des dimensions
print("X_train transformé:", X_train_processed.shape)
print("X_test transformé:", X_test_processed.shape)


📏 ÉTAPE 6: PREPROCESSING
------------------------------
X_train transformé: (3874, 55)
X_test transformé: (969, 55)


In [12]:
# ============================================================================
# ÉTAPE 7: ENTRAÎNEMENT DES MODÈLES AVEC GRIDSEARCH
# ============================================================================
print("\n🎯 ÉTAPE 7: ENTRAÎNEMENT DES 3 MODÈLES")
print("-" * 30)

# Dictionnaire pour stocker les résultats
results = {}


🎯 ÉTAPE 7: ENTRAÎNEMENT DES 3 MODÈLES
------------------------------


In [13]:
# === MODÈLE 1: LINEAR REGRESSION ===
print(f"\n🔵 Modèle 1: Linear Regression")

# Cross-validation pour évaluer le modèle
cv_scores_lr = cross_val_score(LinearRegression(), X_train_processed, y_train, cv=5, scoring='r2')

print(f"   • Cross-validation R² scores: {cv_scores_lr}")
print(f"   • Moyenne CV: {cv_scores_lr.mean():.4f} ± {cv_scores_lr.std():.4f}")

# Entraînement final
lr_model = LinearRegression()
lr_model.fit(X_train_processed, y_train)
lr_pred = lr_model.predict(X_test_processed)

# Métriques
lr_r2 = r2_score(y_test, lr_pred)
lr_rmse = np.sqrt(mean_squared_error(y_test, lr_pred))
lr_mae = mean_absolute_error(y_test, lr_pred)

results['Linear Regression'] = {
    'model': lr_model,
    'r2': lr_r2,
    'rmse': lr_rmse,
    'mae': lr_mae,
    'cv_mean': cv_scores_lr.mean(),
    'cv_std': cv_scores_lr.std(),
    'predictions': lr_pred
}

print(f"   ✅ Résultats Linear Regression:")
print(f"      • R²: {lr_r2:.4f}")
print(f"      • RMSE: {lr_rmse:.2f}€")
print(f"      • MAE: {lr_mae:.2f}€")


🔵 Modèle 1: Linear Regression
   • Cross-validation R² scores: [0.70878837 0.62866517 0.70400578 0.71701575 0.74874643]
   • Moyenne CV: 0.7014 ± 0.0396
   ✅ Résultats Linear Regression:
      • R²: 0.6937
      • RMSE: 17.96€
      • MAE: 12.12€


In [14]:
# === MODÈLE 2: RANDOM FOREST AVEC GRIDSEARCH ===
print(f"\n🌲 Modèle 2: Random Forest + GridSearch")

# Définition des paramètres à tester (version réduite pour rapidité)
param_grid_rf = {
    'n_estimators': [100, 200],              # Nombre d'arbres  
    'max_depth': [10, 15, None],             # Profondeur maximale
    'min_samples_split': [2, 5],             # Échantillons min pour split
    'min_samples_leaf': [1, 2]               # Échantillons min par feuille
}

print(f"   🔍 GridSearch avec {len(param_grid_rf['n_estimators']) * len(param_grid_rf['max_depth']) * len(param_grid_rf['min_samples_split']) * len(param_grid_rf['min_samples_leaf'])} combinaisons")

# GridSearchCV avec validation croisée
rf_grid = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid_rf,
    cv=5,                    # 5-fold cross-validation
    scoring='r2',            # Métrique d'optimisation            
)

# Entraînement (Random Forest n'a pas besoin de normalisation)
print(f"   ⏳ Recherche des meilleurs paramètres...")
rf_grid.fit(X_train_processed, y_train)

# Meilleurs paramètres
best_rf = rf_grid.best_estimator_
print(f"   🏆 Meilleurs paramètres Random Forest:")
for param, value in rf_grid.best_params_.items():
    print(f"      • {param}: {value}")

# Prédictions avec le meilleur modèle
rf_pred = best_rf.predict(X_test_processed)

# Métriques
rf_r2 = r2_score(y_test, rf_pred)
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_pred))
rf_mae = mean_absolute_error(y_test, rf_pred)

results['Random Forest'] = {
    'model': best_rf,
    'r2': rf_r2,
    'rmse': rf_rmse,
    'mae': rf_mae,
    'cv_mean': rf_grid.best_score_,
    'cv_std': 0,  # GridSearch ne donne pas std
    'predictions': rf_pred,
    'best_params': rf_grid.best_params_
}

print(f"   ✅ Résultats Random Forest:")
print(f"      • R²: {rf_r2:.4f}")
print(f"      • RMSE: {rf_rmse:.2f}€")
print(f"      • MAE: {rf_mae:.2f}€")
print(f"      • Best CV Score: {rf_grid.best_score_:.4f}")


🌲 Modèle 2: Random Forest + GridSearch
   🔍 GridSearch avec 24 combinaisons
   ⏳ Recherche des meilleurs paramètres...
   🏆 Meilleurs paramètres Random Forest:
      • max_depth: 15
      • min_samples_leaf: 1
      • min_samples_split: 5
      • n_estimators: 200
   ✅ Résultats Random Forest:
      • R²: 0.7341
      • RMSE: 16.73€
      • MAE: 10.69€
      • Best CV Score: 0.7567


In [15]:
# === MODÈLE 3: XGBOOST AVEC GRIDSEARCH ===

print(f"\n🚀 Modèle 3: XGBoost + GridSearch")
    
    # Paramètres XGBoost optimisés pour débutant (version simplifiée)
param_grid_xgb = {
        'n_estimators': [100, 200],           # Nombre d'arbres
        'max_depth': [4, 6],                  # Profondeur max
        'learning_rate': [0.1, 0.15],        # Taux d'apprentissage
        'subsample': [0.8, 0.9]               # Échantillonnage
    }
    
print(f"   🔍 GridSearch XGBoost avec {len(param_grid_xgb['n_estimators']) * len(param_grid_xgb['max_depth']) * len(param_grid_xgb['learning_rate']) * len(param_grid_xgb['subsample'])} combinaisons")
    
# GridSearchCV pour XGBoost
xgb_grid = GridSearchCV(
        XGBRegressor(random_state=42, eval_metric='rmse'),
        param_grid_xgb,
        cv=5,
        scoring='r2',
    )
    
print(f"   ⏳ Optimisation XGBoost en cours...")
xgb_grid.fit(X_train_processed, y_train)
    
# Meilleurs paramètres
best_xgb = xgb_grid.best_estimator_
print(f"   🏆 Meilleurs paramètres XGBoost:")
for param, value in xgb_grid.best_params_.items():
    print(f"      • {param}: {value}")
    
# Prédictions
xgb_pred = best_xgb.predict(X_test_processed)
    
# Métriques
xgb_r2 = r2_score(y_test, xgb_pred)
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_pred))
xgb_mae = mean_absolute_error(y_test, xgb_pred)
    
results['XGBoost'] = {
        'model': best_xgb,
        'r2': xgb_r2,
        'rmse': xgb_rmse,
        'mae': xgb_mae,
        'cv_mean': xgb_grid.best_score_,
        'cv_std': 0,
        'predictions': xgb_pred,
        'best_params': xgb_grid.best_params_
    }
    
print(f"   ✅ Résultats XGBoost:")
print(f"      • R²: {xgb_r2:.4f}")
print(f"      • RMSE: {xgb_rmse:.2f}€")
print(f"      • MAE: {xgb_mae:.2f}€")
print(f"      • Best CV Score: {xgb_grid.best_score_:.4f}")
    



🚀 Modèle 3: XGBoost + GridSearch
   🔍 GridSearch XGBoost avec 16 combinaisons
   ⏳ Optimisation XGBoost en cours...
   🏆 Meilleurs paramètres XGBoost:
      • learning_rate: 0.15
      • max_depth: 6
      • n_estimators: 100
      • subsample: 0.9
   ✅ Résultats XGBoost:
      • R²: 0.7500
      • RMSE: 16.23€
      • MAE: 10.34€
      • Best CV Score: 0.7657


In [16]:
# ============================================================================
# ÉTAPE 8: COMPARAISON DES MODÈLES
# ============================================================================
print("\n🏆 ÉTAPE 8: COMPARAISON DES MODÈLES")
print("-" * 30)

# Tableau de comparaison amélioré
print(f"{'Modèle':<20} {'R²':<8} {'RMSE':<8} {'MAE':<8} {'CV Score':<10} {'Rang':<6}")
print("-" * 70)

# Classement par R² score
model_ranking = sorted(results.items(), key=lambda x: x[1]['r2'], reverse=True)

for rank, (name, metrics) in enumerate(model_ranking, 1):
    print(f"{name:<20} {metrics['r2']:<8.4f} {metrics['rmse']:<8.2f} {metrics['mae']:<8.2f} {metrics['cv_mean']:<10.4f} #{rank}")

# Sélection du meilleur modèle
best_model_name = model_ranking[0][0]
best_model = results[best_model_name]['model']

print(f"\n🏆 CHAMPION: {best_model_name}")
print(f"   • R² Score: {results[best_model_name]['r2']:.4f}")
print(f"   • RMSE: {results[best_model_name]['rmse']:.2f}€")
print(f"   • MAE: {results[best_model_name]['mae']:.2f}€")
print(f"   • Explication: {results[best_model_name]['r2']*100:.1f}% de la variance des prix")

# Comparaison des améliorations
if len(results) > 1:
    second_best = model_ranking[1][0]
    improvement_r2 = results[best_model_name]['r2'] - results[second_best]['r2']
    improvement_mae = results[second_best]['mae'] - results[best_model_name]['mae']
    
    print(f"\n📈 Amélioration vs {second_best}:")
    print(f"   • R² amélioré de: +{improvement_r2:.4f}")
    print(f"   • MAE réduite de: -{improvement_mae:.2f}€")


🏆 ÉTAPE 8: COMPARAISON DES MODÈLES
------------------------------
Modèle               R²       RMSE     MAE      CV Score   Rang  
----------------------------------------------------------------------
XGBoost              0.7500   16.23    10.34    0.7657     #1
Random Forest        0.7341   16.73    10.69    0.7567     #2
Linear Regression    0.6937   17.96    12.12    0.7014     #3

🏆 CHAMPION: XGBoost
   • R² Score: 0.7500
   • RMSE: 16.23€
   • MAE: 10.34€
   • Explication: 75.0% de la variance des prix

📈 Amélioration vs Random Forest:
   • R² amélioré de: +0.0158
   • MAE réduite de: -0.36€


In [17]:
# ============================================================================
# ÉTAPE 9: ANALYSE DES RÉSULTATS
# ============================================================================

# Importance des features (pour modèles tree-based)
print("\n📊 ÉTAPE 9: ANALYSE DES RÉSULTATS")
print("-" * 30)

# SOLUTION: Utiliser les vrais noms de features depuis ton DataFrame X
correct_feature_names = X.columns.tolist()  # X est ton DataFrame final
print(f"✅ Features utilisées: {len(correct_feature_names)}")
print(f"📋 Liste: {correct_feature_names}")

# Vérification de cohérence
if hasattr(best_model, 'feature_importances_'):
    importances = best_model.feature_importances_
    print(f"📊 Nombre d'importances: {len(importances)}")
    
    if len(correct_feature_names) == len(importances):
        print("✅ Longueurs cohérentes!")
    else:
        print(f"⚠️ MISMATCH: {len(correct_feature_names)} noms vs {len(importances)} importances")

# Importance des features (pour modèles tree-based)
if best_model_name in ['Random Forest', 'XGBoost']:
    print(f"\n🔍 Importance des Features ({best_model_name}):")
    
    if hasattr(best_model, 'feature_importances_'):
        importances = best_model.feature_importances_
        
        # CORRECTION: Vérifier la cohérence avant de créer le DataFrame
        if len(correct_feature_names) == len(importances):
            feature_importance = pd.DataFrame({
                'feature': correct_feature_names,
                'importance': importances
            }).sort_values('importance', ascending=False)
            
            print(feature_importance)
            
            print(f"\n💡 Top 5 features les plus importantes:")
            for i in range(min(5, len(feature_importance))):
                feat = feature_importance.iloc[i]
                print(f"   {i+1}. {feat['feature']}: {feat['importance']:.3f}")
        else:
            # Solution de secours: affichage simple
            print("📊 Affichage des importances par index:")
            for i, imp in enumerate(importances):
                feature_name = correct_feature_names[i] if i < len(correct_feature_names) else f"feature_{i}"
                print(f"   {i+1}. {feature_name}: {imp:.3f}")

elif best_model_name == 'Linear Regression':
    print(f"\n📊 Coefficients Linear Regression:")
    coeffs = pd.DataFrame({
        'feature': feature_names,
        'coefficient': best_model.coef_
    }).sort_values('coefficient', key=abs, ascending=False)
    
    print(coeffs)
    
    print(f"\n💡 Interprétation coefficients:")
    print(f"   • Coefficient positif = augmente le prix")
    print(f"   • Coefficient négatif = diminue le prix")
    print(f"   • Plus la valeur absolue est grande, plus l'impact est fort")

# Analyse des erreurs améliorée
print(f"\n🎯 Analyse des Erreurs ({best_model_name}):")
best_predictions = results[best_model_name]['predictions']
errors = y_test - best_predictions
abs_errors = np.abs(errors)

print(f"   • Erreur moyenne: {errors.mean():.2f}€")
print(f"   • Erreur absolue moyenne: {abs_errors.mean():.2f}€")
print(f"   • Erreur médiane: {np.median(abs_errors):.2f}€")
print(f"   • 90% des prédictions à ±{np.percentile(abs_errors, 90):.2f}€")
print(f"   • 95% des prédictions à ±{np.percentile(abs_errors, 95):.2f}€")

# Pourcentage d'erreur relatif
mean_price = y_test.mean()
relative_error = (abs_errors.mean() / mean_price) * 100
print(f"   • Erreur relative: {relative_error:.1f}% du prix moyen")

# Comparaison avec les autres modèles
if len(results) > 1:
    print(f"\n📊 Comparaison performances:")
    for name, metrics in results.items():
        error_pct = (metrics['mae'] / mean_price) * 100
        print(f"   • {name}: {metrics['mae']:.2f}€ MAE ({error_pct:.1f}% erreur)")

# Analyse des cas d'erreur importante
high_error_threshold = np.percentile(abs_errors, 90)
high_error_indices = np.where(abs_errors > high_error_threshold)[0]

print(f"\n🔍 Analyse des erreurs importantes (>90e percentile = {high_error_threshold:.2f}€):")
print(f"   • Nombre de cas: {len(high_error_indices)} sur {len(y_test)} ({len(high_error_indices)/len(y_test)*100:.1f}%)")

if len(high_error_indices) > 0:
    high_error_prices = y_test.iloc[high_error_indices]
    print(f"   • Prix moyen de ces cas: {high_error_prices.mean():.2f}€")
    print(f"   • Ces erreurs concernent principalement:")
    if high_error_prices.mean() > mean_price * 1.5:
        print(f"     → Voitures premium/luxe (difficiles à prédire)")
    elif high_error_prices.mean() < mean_price * 0.5:
        print(f"     → Voitures très économiques (peu d'exemples)")
    else:
        print(f"     → Voitures avec caractéristiques atypiques")


📊 ÉTAPE 9: ANALYSE DES RÉSULTATS
------------------------------
✅ Features utilisées: 13
📋 Liste: ['mileage', 'engine_power', 'private_parking_available', 'has_gps', 'has_air_conditioning', 'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires', 'model_key', 'fuel', 'paint_color', 'car_type']
📊 Nombre d'importances: 55
⚠️ MISMATCH: 13 noms vs 55 importances

🔍 Importance des Features (XGBoost):
📊 Affichage des importances par index:
   1. mileage: 0.035
   2. engine_power: 0.114
   3. private_parking_available: 0.009
   4. has_gps: 0.050
   5. has_air_conditioning: 0.013
   6. automatic_car: 0.025
   7. has_getaround_connect: 0.052
   8. has_speed_regulator: 0.011
   9. winter_tires: 0.017
   10. model_key: 0.013
   11. fuel: 0.049
   12. paint_color: 0.033
   13. car_type: 0.013
   14. feature_13: 0.014
   15. feature_14: 0.006
   16. feature_15: 0.000
   17. feature_16: 0.013
   18. feature_17: 0.003
   19. feature_18: 0.003
   20. feature_19: 0.006
   21. f

In [18]:
# ============================================================================
# ÉTAPE 10: SAUVEGARDE DU MODÈLE
# ============================================================================
print("\n💾 ÉTAPE 10: SAUVEGARDE")
print("-" * 30)

# Création du package complet à sauvegarder
model_package = {
    'best_model': best_model,
    'best_model_name': best_model_name,
    'preprocessor': preprocessor,
    'label_encoders': label_encoders,
    'feature_names': feature_names,
    'results': results,
    'training_stats': {
        'n_samples': len(X),
        'n_features': len(feature_names),
        'target_mean': y.mean(),
        'target_std': y.std()
    }
}

# Sauvegarde
joblib.dump(model_package, 'getaround_model_simple.joblib')
print(f"✅ Modèle sauvegardé: getaround_model_simple.joblib")


💾 ÉTAPE 10: SAUVEGARDE
------------------------------
✅ Modèle sauvegardé: getaround_model_simple.joblib


In [21]:
# ============================================================================
# ÉTAPE 11: TEST DE PRÉDICTION (ADAPTÉ AUX NOUVELLES FEATURES)
# ============================================================================
print("\n🔮 ÉTAPE 11: TEST DE PRÉDICTION")
print("-" * 30)

def predict_price(model_package, car_features):
    """
    Prédit le prix d'une voiture - VERSION ADAPTÉE AUX NOUVELLES FEATURES
    
    car_features = {
        'model_key': 'Citroën',
        'mileage': 50000,
        'engine_power': 120,
        'fuel': 'diesel',
        'paint_color': 'black',
        'car_type': 'hatchback',
        'private_parking_available': True,
        'has_gps': True,
        'has_air_conditioning': True,
        'automatic_car': False,
        'has_getaround_connect': True,
        'has_speed_regulator': True,
        'winter_tires': False
    }
    """
    
    print("🔧 CALCUL DES FEATURES POUR PRÉDICTION")
    print("=" * 40)
    
    # Extraction des composants du modèle
    model = model_package['best_model']
    scaler = model_package.get('scaler')
    label_encoders = model_package['label_encoders']
    best_model_name = model_package['best_model_name']
    
    # ========================================================================
    # CALCUL DES FEATURES NUMÉRIQUE
    # ========================================================================
    
    # Features numériques directes (converties en float pour être sûr)
    numeric_features = ['mileage', 'engine_power',
                       'private_parking_available', 'has_gps', 
                       'has_air_conditioning', 'automatic_car',
                       'has_getaround_connect', 'has_speed_regulator', 
                       'winter_tires']
    
    numeric_values = []
    
    for feature in numeric_features:
        if feature in ['private_parking_available', 'has_gps', 'has_air_conditioning', 
                      'automatic_car', 'has_getaround_connect', 'has_speed_regulator', 'winter_tires']:
            # Convertir les booléens en entiers
            value = int(car_features[feature])
        else:
            # mileage et engine_power restent tels quels
            value = car_features[feature]
        
        numeric_values.append(value)
        print(f"✅ {feature}: {value}")
    
    # ========================================================================
    # ENCODAGE DES FEATURES CATÉGORIELLES
    # ========================================================================
    
    categorical_features = ['model_key', 'fuel', 'paint_color', 'car_type']
    categorical_values = []
    
    for feature in categorical_features:
        try:
            encoded_value = label_encoders[feature].transform([car_features[feature]])[0]
            categorical_values.append(encoded_value)
            print(f"✅ {feature} encoded: {encoded_value} ('{car_features[feature]}')")
        except ValueError:
            # Si la valeur n'était pas dans l'entraînement, utiliser une valeur par défaut
            print(f"⚠️ {feature} '{car_features[feature]}' inconnu, utilisation valeur par défaut")
            encoded_value = 0  # Valeur par défaut
            categorical_values.append(encoded_value)
        except KeyError:
            print(f"❌ Encodeur pour '{feature}' non trouvé!")
            categorical_values.append(0)
    
    # ========================================================================
    # CRÉATION DU VECTEUR DE FEATURES (ORDRE EXACT DE TON ENTRAÎNEMENT)
    # ========================================================================
    
    # L'ordre doit correspondre exactement à ton X dans l'entraînement:
    # X = pd.concat([X_numeric, X_categorical], axis=1)
    # Donc: numeric_features + categorical_features
    
    features_vector = numeric_values + categorical_values
    
    print(f"\n📊 VECTEUR DE FEATURES FINAL:")
    all_feature_names = numeric_features + categorical_features
    
    for i, (name, value) in enumerate(zip(all_feature_names, features_vector), 1):
        print(f"   {i}. {name}: {value}")
    
    print(f"\n✅ Nombre total de features: {len(features_vector)}")
    
    # ========================================================================
    # PRÉDICTION
    # ========================================================================
    
    # Reshape pour sklearn
    features_array = np.array(features_vector).reshape(1, -1)
    
    try:
        # Prédiction selon le type de modèle
        if best_model_name == 'Linear Regression' and scaler is not None:
            features_scaled = scaler.transform(features_array)
            prediction = model.predict(features_scaled)[0]
        else:
            prediction = model.predict(features_array)[0]
        
        final_price = max(10, round(prediction, 2))  # Prix minimum 10€
        
        print(f"\n🎯 RÉSULTAT:")
        print(f"   Prix prédit: {final_price}€/jour")
        
        return final_price
        
    except ValueError as e:
        print(f"❌ ERREUR DE PRÉDICTION: {e}")
        print(f"📊 Shape du vecteur: {features_array.shape}")
        
        # Debug supplémentaire
        if hasattr(model, 'get_booster'):
            expected_features = model.get_booster().num_features()
        elif hasattr(model, 'n_features_in_'):
            expected_features = model.n_features_in_
        else:
            expected_features = "N/A"
            
        print(f"📊 Features attendues par le modèle: {expected_features}")
        print(f"📊 Features fournies: {len(features_vector)}")
        
        return None

# ============================================================================
# FONCTION DE VALIDATION DES FEATURES
# ============================================================================

def validate_features_compatibility(model_package, X):
    """
    Valide que les features sont compatibles avec le modèle
    """
    print("🔍 VALIDATION DE COMPATIBILITÉ DES FEATURES")
    print("=" * 45)
    
    model = model_package['best_model']
    label_encoders = model_package['label_encoders']
    
    # Vérifier le nombre de features
    if hasattr(model, 'get_booster'):
        expected_features = model.get_booster().num_features()
    elif hasattr(model, 'n_features_in_'):
        expected_features = model.n_features_in_
    else:
        expected_features = "N/A"
    
    actual_features = X.shape[1]
    
    print(f"📊 Features attendues par le modèle: {expected_features}")
    print(f"📊 Features dans ton dataset: {actual_features}")
    
    if expected_features != "N/A" and expected_features == actual_features:
        print("✅ Nombre de features compatible!")
    else:
        print("⚠️ Nombre de features incompatible!")
    
    # Vérifier les encodeurs disponibles
    print(f"\n🏷️ Encodeurs disponibles:")
    for name, encoder in label_encoders.items():
        print(f"   • {name}: {len(encoder.classes_)} classes")
        print(f"     Classes: {list(encoder.classes_)[:5]}{'...' if len(encoder.classes_) > 5 else ''}")
    
    # Vérifier l'ordre des features
    print(f"\n📋 Ordre des features dans le dataset:")
    feature_names = X.columns.tolist()
    for i, name in enumerate(feature_names, 1):
        print(f"   {i}. {name}")
    
    return expected_features == actual_features

# ============================================================================
# TEST DE LA FONCTION ADAPTÉE
# ============================================================================

# Voiture de test (avec paint_color ajouté)
test_car = {
    'model_key': 'Citroën',
    'mileage': 50000,
    'engine_power': 120,
    'fuel': 'diesel',
    'paint_color': 'black',
    'car_type': 'hatchback',
    'private_parking_available': True,
    'has_gps': True,
    'has_air_conditioning': True,
    'automatic_car': False,
    'has_getaround_connect': True,
    'has_speed_regulator': True,
    'winter_tires': False
}

# Charger les objets
model = model_package['best_model']
preprocessor = model_package['preprocessor']

# Convertir en DataFrame
test_df = pd.DataFrame([test_car])

# Appliquer le préprocessing
X_processed = preprocessor.transform(test_df)

# Prédire
prediction = model.predict(X_processed)[0]
print(f"🎯 Prix prédit : {round(prediction, 2)}€/jour")


🔮 ÉTAPE 11: TEST DE PRÉDICTION
------------------------------
🎯 Prix prédit : 142.39999389648438€/jour


In [22]:
# ============================================================================
# RÉSUMÉ FINAL
# ============================================================================
print(f"\n" + "="*60)
print(f"🎉 PIPELINE ML TERMINÉ AVEC SUCCÈS!")
print(f"="*60)

print(f"\n📊 RÉSUMÉ DES PERFORMANCES:")
print(f"   🏆 Meilleur modèle: {best_model_name}")
print(f"   📈 R² Score: {results[best_model_name]['r2']:.4f}")
print(f"   💰 RMSE: {results[best_model_name]['rmse']:.2f}€")
print(f"   🎯 Précision: ±{results[best_model_name]['mae']:.2f}€ en moyenne")

print(f"\n🔧 FEATURES UTILISÉES: {len(feature_names)}")
for i, feature in enumerate(feature_names, 1):
    print(f"   {i}. {feature}")

print(f"\n💾 MODÈLE SAUVEGARDÉ: getaround_model_simple.joblib")


🎉 PIPELINE ML TERMINÉ AVEC SUCCÈS!

📊 RÉSUMÉ DES PERFORMANCES:
   🏆 Meilleur modèle: XGBoost
   📈 R² Score: 0.7500
   💰 RMSE: 16.23€
   🎯 Précision: ±10.34€ en moyenne

🔧 FEATURES UTILISÉES: 13
   1. mileage
   2. engine_power
   3. private_parking_available
   4. has_gps
   5. has_air_conditioning
   6. automatic_car
   7. has_getaround_connect
   8. has_speed_regulator
   9. winter_tires
   10. model_key
   11. fuel
   12. paint_color
   13. car_type

💾 MODÈLE SAUVEGARDÉ: getaround_model_simple.joblib
